In [2]:
import os
import pandas as pd
import numpy as np
import glob

In [29]:
sub = 'sub-05'
folder = 'scene_perception'
task = 'scene-perception'

main_dir = '/home/sshankar/ibc/analysis_pipeline/ibc_main/neurospin_data/info/'
datadir = os.path.join(main_dir, sub, folder)
os.chdir(datadir)
csv_files = sorted(glob.glob('*' + task + '*.csv'))
tsv_files = []
i = 1
tsv_files.append(task + '_' + sub + '_run-0' + str(i+1) + '.tsv')

In [30]:
file_df = pd.read_csv(os.path.join(datadir, csv_files[i]))

In [31]:
# In the csv file extracted from the E-Prime txt file, 
# run onset is the 'GetReady.OffsetTime' column of last row, 
# and all other columns of this row are empty.
run_start = (file_df.loc[~np.isnan(file_df['Fix.OnsetTime'])])['Fix.OnsetTime']
run_start = run_start.values[0]

In [32]:
run_start

52088.0

In [33]:
# There are 3 events per trial: Fix, Stimulus, ITI. Each trial has all 3 events
# Fix onsets are in Fix.OnsetTime
fix_onsets = (file_df.loc[~np.isnan(file_df['Fix.OnsetTime'])])['Fix.OnsetTime']
n_trials = len(fix_onsets)
fix_types = ['fix'] * n_trials 

# Stimulus onsets are in 'Stimulus.OnsetTime'
stim_onsets = (file_df.loc[~np.isnan(file_df['Stimulus.OnsetTime'])])['Stimulus.OnsetTime']
fix_durations = (stim_onsets - fix_onsets)/1000

# Trial_types are in 'Condition'
stim_types = (file_df.loc[~np.isnan(file_df['Stimulus.OnsetTime'])])['Condition']
accuracy = (file_df.loc[~np.isnan(file_df['Stimulus.OnsetTime'])])['Stimulus.ACC']
stim_types[np.logical_and(stim_types=='scn_pos', accuracy==1)] = 'scene_possible_correct'
stim_types[np.logical_and(stim_types=='scn_pos', accuracy==0)] = 'scene_possible_incorrect'
stim_types[np.logical_and(stim_types=='scn_imp', accuracy==1)] = 'scene_impossible_correct'
stim_types[np.logical_and(stim_types=='scn_imp', accuracy==0)] = 'scene_impossible_incorrect'
stim_types[np.logical_and(stim_types=='dot_pos_l', accuracy==1)] = 'dot_easy_left_correct'
stim_types[np.logical_and(stim_types=='dot_pos_r', accuracy==1)] = 'dot_easy_right_correct'
stim_types[np.logical_and(stim_types=='dot_pos_l', accuracy==0)] = 'dot_easy_left_incorrect'
stim_types[np.logical_and(stim_types=='dot_pos_r', accuracy==0)] = 'dot_easy_right_incorrect'
stim_types[np.logical_and(stim_types=='dot_imp_l', accuracy==1)] = 'dot_hard_left_correct'
stim_types[np.logical_and(stim_types=='dot_imp_r', accuracy==1)] = 'dot_hard_right_correct'
stim_types[np.logical_and(stim_types=='dot_imp_l', accuracy==0)] = 'dot_hard_left_incorrect'
stim_types[np.logical_and(stim_types=='dot_imp_r', accuracy==0)] = 'dot_hard_right_incorrect'

# In this task, stimulus/trial durations are 5 s
stim_durations = np.ones(n_trials) * 5

# ITI onsets are in ITI.OnsetTime
iti_onsets = (file_df.loc[~np.isnan(file_df['ITI.OnsetTime'])])['ITI.OnsetTime']
iti_durations = (file_df.loc[~np.isnan(file_df['ITI.OnsetTime'])])['ITI']
iti_durations = iti_durations/1000
iti_types = ['iti'] * n_trials

/home/sshankar/.local/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/home/sshankar/.local/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/home/sshankar/.local/lib/python3.6/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/sshanka

In [34]:
# Interleave the onsets, durations and types of the 3 events
onsets_list = [fix_onsets, stim_onsets, iti_onsets]
onsets = [val for tup in zip(*onsets_list) for val in tup]

durations_list = [fix_durations, stim_durations, iti_durations]
durations = [val for tup in zip(*durations_list) for val in tup]

types_list = [fix_types, stim_types, iti_types]
trial_types = [val for tup in zip(*types_list) for val in tup]

In [35]:
# Create a DataFrame to save to events file
df = pd.DataFrame({'onset':(onsets-run_start)/1000, 'duration':durations, 'trial_type':trial_types})
df.to_csv(os.path.join(datadir, tsv_files[0]), sep='\t', float_format='%0.3f', index=False)